In [3]:
from ifc_openseespy_linker.core import IFCtoOpenSeesConverter

converter = IFCtoOpenSeesConverter()
converter.load_ifc('sample_models/CASA TIPO 11111.ifc')
structural_elements = converter.extract_structural_elements()
print(f"Extracted elements: {len(structural_elements)}")
print("Element types:", {elem['type'] for elem in structural_elements.values()})
converter.visualize_model()

Extracted elements: 89
Element types: {'IfcColumn', 'IfcFooting', 'IfcBeam', 'IfcSlab'}
